# Create all Author and Publication Objects

In [3]:
# TODO: import `load_data`, `get_author_names_list`, and `extract_names` from process_names.ipynb: DONE

from preprocessing.process_names import load_data, get_author_names_list, extract_names
from preprocessing.author import Author
import pandas as pd

## Create an Author for ***every*** author listed in the first publication

1. Get the publication from first row
2. Create a Publication object and add it to a list of publications
3. Get the author list as a string
4. Transform the string into a list
5. Extract the first, middle, and last name to create a single author object for every author in list
6. Add the Publication to the the Author's publication attribute

All the steps are the same, except we want to turn the last step into a loop so we turn every author in the list into an author object.

In [8]:
publication_list = [] # list of Publication objects

In [7]:
class Publication:
    def __init__(self, id, title, doi):
        self.id = id
        self.title = title
        self.doi = doi

In [11]:
# 2. Get the author list as a string
author_names = single_publication['author_names']
author_names

"['Geoff Podger', 'Robert Power', 'Shane Seaton', 'Ang Yang']"

In [12]:
# 3. Transform the string into a list
author_names_list = get_author_names_list(author_names)
author_names_list

['Geoff Podger', 'Robert Power', 'Shane Seaton', 'Ang Yang']

In [13]:
# 4. For each author name in list, extract the first, middle, and last name to create a single author object

author_list = [] # list of Author objects

In [ ]:
for author_string in author_names_list:
    # Extract first, middle, and last name from each author string
    last_name, first_name, middle_name, middle_name2, middle_name3 = extract_names(author_string)

    # Create an Author object
    author = Author(last_name, middle_name, last_name)

    # TODO: add the publication to the author's publication attribute
    author.publications.append(publication)
   
    # Add the author to the author list
    author_list.append(author)

In [ ]:
author_list

## Create an Author for every author listed in the first publication ***and*** add an email if that author is also the contact author

1. Create an Author object for the contact author
2. Write a function that compares the author to the contact author to see if they are the same
3. If they are the same, add the contact author email to the author

In [ ]:
## Check if the contact author matches a sigle author

In [ ]:
single_publication = df.loc[1]
single_publication

In [ ]:
# get contact author name from single_publication
contact_author_name = single_publication['contact_author_name']
contact_author_name

In [ ]:
# use a function to extract the first, middle, and last names of the contact author
contact_first, contact_middle, contact_last = extract_names(contact_author_name)
contact_first, contact_middle, contact_last

In [ ]:
single_publication['contact_email']


In [ ]:
df_small['contact_email']

In [ ]:
# get the contact email for the contact author
contact_author_email = single_publication['contact_email']
contact_author_email

In [ ]:
# get the contact email for the contact author
contact_author_email = single_publication['contact_email']
contact_author_email

In [ ]:
# create an Author object for the contact author

contact_author = Author(contact_first, contact_middle, contact_last, contact_author_email)
contact_author

Let's save the author that matchs the contact author.

In [ ]:
fourth_author = author_list[2]
fourth_author

In [ ]:
fourth_author.first == contact_author.first \
    and fourth_author.middle == contact_author.middle \
    and fourth_author.last == contact_author.last

Write a function that compares an author to the contact author and returns True if there is a match. We will call this function `__eq__`, because it is a special dunder method that allows us to compare two objects with a double equal sign (`==`). We will see how this function works when we integrate it into our class later. 

In [ ]:
def __eq__(author, contact_author):

    return author.first == contact_author.first and author.middle == contact_author.middle and author.last == contact_author.last
   


In [ ]:
__eq__(author_list[5], contact_author)

Does it return `True`? Now let's try one we know is not a match and should return `False`. 

In [ ]:
author_list[5]

In [ ]:
__eq__(author_list[0], contact_author)

Finally, when we do find a match between an author and a contact author, we want to use the contact author information to add information to the author. We could do it like this...

In [ ]:
if __eq__(fourth_author, contact_author):
    fourth_author.email = contact_author.email

... and we can see that it works because the email is now there.

In [ ]:
fourth_author

In [ ]:
contact_author

But later down the line, we might want to also update the middle name or something as well. So we need to write a function to accomodate these future changes. Also, we want to add some validation to check that the two authors do actually share a name before we merge their information together.

In [ ]:
def add_contact_author_info(author, contact_author):
    if not __eq__(author, contact_author):
        raise Exception('not a match')
    else: 
        #author_list.append(contact_author_email)
        author.email = contact_author.email

Does it work? Let's try it for an author that we known matchs...

In [ ]:
add_contact_author_info(fourth_author, contact_author)

In [ ]:
fourth_author

And one that doesn't. The following code should throw an error...

In [ ]:
add_contact_author_info(author_list[0], contact_author)

Okay! Now the only thing left to do is to put this all together in our loop. Take what you had before, but add a few lines to check if the new author and the contact author are the same. If they are, you need to add the contact author info to the new author. Use the commented code below to do it. 

**Don't forget to use** `__eq__` **and** `add_contact_author_info`**!**

In [ ]:
for author_string in author_names_list:
    # extract first, middle, and last name from each author string
    # TODO: replace the code below with a function: DONE
    last_name, first_name, middle_name1, middle_name2, middle_name3 = extract_names(author_string)
    
    # create an Author object
    author = Author(last_name, first_name, middle_name)
    
    # TODO: add publication to author's publication list: DONE
    author.publications.append(publication)
    
    if not __eq__(author, contact_author): 
        print ("Not a match")
    else:
        # add the author to the author list
        add_contact_author_info(author, contact_author)
        print("Done")

## ***For every publication...*** create a Publication and an Author for every author in the list and add an email if that author is also the contact author


The next step is to run the loop above for every single row in `df_small`. If you want, you can get started on that below.

In [4]:
df = load_data()

In [5]:
' '.strip()

''

In [52]:
publication_list = []
author_list = []

for index, row in df.iterrows():

    #If title or contact_email exists
    if (row['title'] or row['contact_email']):
            
        # create a new publication object- DONE
        publication = Publication(id=row['id'], title=row['title'], doi=row['doi'])
        
        # add the publication to the list- DONE
        publication_list.append(publication)

        
      #  if pd.isna(row["author_names"]):
            
        
      
    
        #Create contact author
        
        contact_name = row["contact_author_name"]

       
           
       
        
        if not pd.isna(row["contact_author_name"]):
            contact_last, contact_first, contact_middle, _, _ = extract_names(contact_name)
            contact_author = Author(contact_last, contact_first, contact_middle, emails = [row["contact_email"]]) 


        if not pd.isna(row['author_names']):
            test = row['author_names'].strip('[\'] ')
            
        if (test == "") or not pd.isna(row['author_names']):
            # create an Author object for every author belonging to the publication - DONE
            author_names_list = [contact_name]

        else: 
            author_names_list = get_author_names_list(row['author_names'])

        for author_name in author_names_list:
            # Extract first, middle, and last name from each author string
            try:
                last_name, first_name, middle_name1, middle_name2, middle_name3 = extract_names(author_name)
            except Exception as e:
                print(author_name, "+", contact_name)
                print(type(author_name))
                print("!")
                raise e
            
            # Create an Author object
            author = Author(last_name, first_name, middle_name1)
       
            # Add the publication to the Author's list of publications - DONE
            author.publications.append(publication)
            
            # if that author is also the contact author, add an email - DONE
            if (author == contact_author):
                print ("True", author)
                author.add_contact_author_info(contact_author)
                
                
            # add the Author to the list of Authors - DONE
            author_list.append(author)

###TO DO: No contact author or author name....

True Ang None Yang
True Lei None Zhang
True Mehdi None Saqalli
True N S Tay
True FrÃ©dÃ©ric None Thomas
True Peter None Todd
True Christopher J Topping
True Christopher J Topping
True Mark A Ratner
True K None Ueda
True D C Walker
True Robert None Walker
True S L Wang
True Ravi None Bhavnani
True Ravi None Bhavnani
True Sukaina None Bharwani
True Massimo None Bernaschi
True Matthew None Berman
True Alexander None Bentley
True Tony None White
True Tony None White
True Francesc S Beltran
True Olivier None Barreteau
True Catherine None Beachemin
True Michael None Batty
True Iqbal None Adjali
True Roland None Barthel
True Eric None Fisher
True Olivier None Barreteau
True Caesar None Saloma
True Alassane None Bah
True Raul None Bagni
True Johannes None Wohlmuth
True Robert None Axelrod
True Ioannis N Athanasiadis
True Sundar None Srinivasan
True Thomas None Deisboeck
True Takaya None Arita
True Peter None Andras
True Li None An
True David None Levinson
True Li None An
True J Alvarez- Ramire

AttributeError: 'float' object has no attribute 'replace'

In [47]:
test = row['author_names'].strip('[\'] ')
test

''

In [ ]:
for publication in publication_list:
    print(publication)
for author in author_list:
    print(author)

In [ ]:
# Print the created publications and authors
for publication in publication_list:
    print(publication.__dict__)

for author in author_list:
    print(author.__dict__)

## For every publication create a Publication and an Author for every author in the list and add an email if that author is also the contact author. 
## **If the author exists already, use the new Author object to update the old one. Otherwise add the new Author to the list of Authors.**

In [ ]:
class Author:
    def __init__(self, first, middle, last, email=None, publications=[]):
        self.first = first
        self.middle = middle
        self.last = last
        self.email = email
        self.publications = publications
def __eq__(author, contact_author):

    return author.first == contact_author.first and author.middle == contact_author.middle and author.last == contact_author.last

def add_contact_author_info(author, contact_author):
    if not __eq__(author, contact_author):
        raise Exception('not a match')
    else: 
        #author_list.append(contact_author_email)
        author.email = contact_author.email


for index, row in df.iterrows():
    # Create a new publication object
    publication = Publication(id=row['id'], title=row['title'], doi=row['doi'])

    # Create an Author object for the contact author
    contact_name = row["contact_author_name"]
    contact_first_name, contact_middle_name, contact_last_name, _, _ = extract_names(contact_name)
    contact_author = Author(contact_first_name, contact_middle_name, contact_last_name, row["contact_email"])

    for author_name in get_author_names_list(row['author_names']):
        # Extract first, middle, and last name from each author string
        first_name, middle_name, last_name = extract_names(author_name)

        # Create an Author object
        author = Author(last_name, first_name, middle_name)

        # Add the publication to the Author's list of publications
        author.publications.append(publication)

        # if that author is also the contact author, add an email - DONE
        if __eq__(author, contact_author):
            print ("True", author)
            add_contact_author_info(author, contact_author)

       """ # Check if the author already exists in the author_list
        existing_author = None
        for existing_author in author_list:
            if __eq__(existing_author, author):
              
                # Update the existing Author object with the new information
                add_contact_author_info(existing_author, author) 
                break
            else:
                # Add the Author to the list of Authors
                author_list.append(author) """

    # Add the publication to the list
    publication_list.append(publication)